In [1]:
!pip install transformers

     |████████████████████████████████| 1.5MB 12.6MB/s 
     |████████████████████████████████| 2.9MB 49.3MB/s 
     |████████████████████████████████| 890kB 49.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b0488d1588e0350b0e3084545b4e1edba8933195bb9665f03f4fad76eb9d633c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

In [3]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Text with symptoms in it

In [5]:
text  = "Pain or discomfort in one or both arms, the back, neck, jaw or stomach, Shortness of breath with or without chest discomfort, Other signs such as breaking out in a cold sweat, nausea or lightheadedness are symptoms of [MASK] [SEP]"
text = "[MASK] attack is caused by blockage of blood"
text = "[MASK] attack has following symptoms: [MASK] in urine, [MASK] in the lower back."

#Using a pre-trained bert for maksed LM to predict the disease, given tetx with symptoms in it.

In [6]:
# Predict tokens

def bertmlm(text):
  tokenized_text = tokenizer.tokenize(text)
  #tokenized_text[masked_index] = '[MASK]'
  mask = []
  for i, m in enumerate(tokenized_text):
    if m == '[MASK]':
      mask.append(i)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [0]*len(indexed_tokens)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  model_mlm = BertForMaskedLM.from_pretrained('bert-base-uncased')
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  tokens_tensor = tokens_tensor.to('cuda')
  segments_tensors = segments_tensors.to('cuda')
  model_mlm.to('cuda')
  model_mlm.eval()
  with torch.no_grad():
      outputs = model_mlm(tokens_tensor, token_type_ids=segments_tensors)
      predictions = outputs[0]
  print(tokenized_text)
  for i in mask:
    predicted_index = torch.argmax(predictions[0, i]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    print(predicted_index,predicted_token)

# Loading the disease symptoms data,

In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv("diseases_texts.csv")
df

,symptoms,disease
0,what are the symptoms of hemorrhagic septicemi...,hemorrhagic septicemia
1,what are the symptoms of jaundice? s that late...,jaundice
2,what are the symptoms of drowning? drowning is...,drowning
3,what are the symptoms of iron-deficiency anemi...,iron-deficiency anemia
4,what are the symptoms of mixed connective tiss...,mixed connective tissue disease
...,...,...
2049,what are the symptoms of patau syndrome? male ...,patau syndrome
2050,what are the symptoms of lambert–eaton myasthe...,lambert–eaton myasthenic syndrome
2051,what are the symptoms of reward system? addict...,reward system
2052,what are the symptoms of treponema pallidum? t...,treponema pallidum


In [8]:
data = df["symptoms"].tolist()

# Preparing the data for fine tuninin the bert for masked model. Adding mask tokens in the data.

In [9]:
def get_masked_input_and_labels(encoded_texts):
    # 15% BERT masking
    inp_mask = np.random.rand(*encoded_texts.shape) < 0.20
    # Do not mask special tokens
    inp_mask[encoded_texts <= 2] = False
    # Set targets to -1 by default, it means ignore
    labels = -1 * np.ones(encoded_texts.shape, dtype=int)
    # Set labels for masked tokens
    labels[inp_mask] = encoded_texts[inp_mask]

    # Prepare input
    encoded_texts_masked = np.copy(encoded_texts)
    # Set input to [MASK] which is the last token for the 90% of tokens
    # This means leaving 10% unchanged
    inp_mask_2mask = inp_mask & (np.random.rand(*encoded_texts.shape) < 0.90)
    encoded_texts_masked[
        inp_mask_2mask
    ] = mask_token_id  # mask token is the last in the dict

    # Set 10% to a random token
    inp_mask_2random = inp_mask_2mask & (np.random.rand(*encoded_texts.shape) < 1 / 9)
    encoded_texts_masked[inp_mask_2random] = np.random.randint(
        3, mask_token_id, inp_mask_2random.sum()
    )

    # Prepare sample_weights to pass to .fit() method
    sample_weights = np.ones(labels.shape)
    sample_weights[labels == -1] = 0

    # y_labels would be same as encoded_texts i.e input tokens
    y_labels = np.copy(encoded_texts)

    return torch.tensor(encoded_texts_masked), torch.tensor(y_labels), sample_weights

Setting the max token length

In [10]:
tokenizer.model_max_length = 128

In [17]:
#len(encoded['input_ids'][0])

Data loader essentials

In [11]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

#train_dataset = Dataset(train_encodings, train_labels)

In [12]:
from torch.utils.data import DataLoader
from transformers import  AdamW
from tqdm import tqdm
import time

In [14]:
device = torch.device('cuda')
mask_token_id = 103

In [16]:
encoded = tokenizer(text, padding = True, truncation=True)
get_masked_input_and_labels(torch.tensor(encoded['input_ids']))

(tensor([   64,   103,  2886,  2038,  2206,  8030,  1024,   103,  1999, 17996,
          1010,   103,  1999,   103,  2896,  2067,  1012,   102]),
 tensor([  101,   103,  2886,  2038,  2206,  8030,  1024,   103,  1999, 17996,
          1010,   103,  1999,  1996,  2896,  2067,  1012,   102]),
 array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0.]))

In [21]:
def train(data):
  mask_token_id = 103
  encoded = tokenizer(data, padding=True, truncation=True)
  encoded['input_ids'], y, w = get_masked_input_and_labels(torch.tensor(encoded['input_ids']))

  train_dataset = Dataset(encoded, y)
  train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
  optim = AdamW(model.parameters(), lr=5e-5)

  model.to(device)
  model.train()
  for epoch in tqdm(range(5)):
      start = time.time()
      for batch in (train_loader):
          optim.zero_grad()
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['labels'].to(device)
          outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
          loss = outputs[0]
          loss.backward()
          optim.step()
      print(time.time() - start)
      print(loss.item())

  return model

# Fine tuning bert for masked model

In [22]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model_fine = train(data)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/i

61.040332555770874
0.4186389446258545



 40%|████      | 2/5 [02:04<03:05, 61.83s/it]

63.59367656707764
0.3388884961605072



 60%|██████    | 3/5 [03:08<02:04, 62.49s/it]

64.00246238708496
0.16335897147655487



 80%|████████  | 4/5 [04:13<01:03, 63.14s/it]

64.64537835121155
0.1364319920539856



100%|██████████| 5/5 [05:18<00:00, 63.64s/it]

64.7859616279602
0.1591934859752655


In [69]:
#text  = "[SEP] Pain or discomfort in one or both arms, the back, neck, jaw or stomach, Shortness of breath with or without chest discomfort, Other signs such as breaking out in a cold sweat, nausea or lightheadedness are symptoms of [MASK]"
#text = "[SEP] I got injured in an accident, I need your [MASK]"
#text = "[SEP] [MASK] cancer have the following symptoms pain or discomfort in one or both arms, the back, neck, jaw or stomach, Shortness of breath with or without chest discomfort "

# Chekcing the performance of the fine tuned model.

In [23]:

def bertfine(model, text):
  model.eval()
  tokenizer.model_max_length = 128
  tokenized_text = tokenizer.encode_plus(text, padding=True, truncation=True)

  # Predict all tokens
  masked_index = 11
  #tokenized_text = tokenizer.tokenize(text)
  #tokenized_text[masked_index] = '[MASK]'
  mask = []
  for i, m in enumerate(tokenizer.convert_ids_to_tokens(tokenized_text["input_ids"])):
    if m == '[MASK]':
      mask.append(i)

  print(tokenizer.convert_ids_to_tokens(tokenized_text["input_ids"]))
  #indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text["input_ids"])
  indexed_tokens = tokenized_text["input_ids"]
  segments_ids = [0]*len(indexed_tokens)
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  #model = BertForMaskedLM.from_pretrained('bert-base-uncased')

  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])

  tokens_tensor = tokens_tensor.to('cuda')
  segments_tensors = segments_tensors.to('cuda')

  with torch.no_grad():
      outputs = model(tokens_tensor, token_type_ids=segments_tensors)
      predictions = outputs[0]

  for i in mask:
    predicted_index = torch.argmax(predictions[0, i]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    print(predicted_index,predicted_token)

In [24]:
text = "woman with [MASK] [MASK], signs and symptoms that are related to the infection are mainly:  high fever, usually above 101 °fahrenheit|f / 38 °c  chills  severe abdominal pain and/or cramping /or strong perineum|perineal pressure  beginning miscarriage symptoms (heavy bleeding and or cramping) that suddenly stops and does not resume  prolonged or heavy vaginal bleeding  foul-smelling vaginal discharge backache or heavy back pressure  a cold or urinary tract infection may mimic many of the symptoms."

In [25]:
texts = ["Blood in the urine, pain while passing urine, pain in the lower back, nausea and vomiting are signs or symptoms of [MASK]",
        "Blood in urine, pain in the left side lower back, nausea and vomiting are symptoms of [MASK] disease",
        "[MASK] pain, swelling of the [MASK], vomiting are symptoms of Appendicites.",
        "[MASK] disease will have following symptoms, blood in urine, pain  in the lower back, [MASK] and vomiting."]
#text = 
#text = 
#text = 

In [27]:
for text in texts:
  print("Bert")
  print("="*50)
  bertmlm(text)
  print()
  print()
  print("Fine Tuned")
  print("="*50)
  bertfine(model_fine, text)
  print()

Bert


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['blood', 'in', 'the', 'urine', ',', 'pain', 'while', 'passing', 'urine', ',', 'pain', 'in', 'the', 'lower', 'back', ',', 'nausea', 'and', 'vomiting', 'are', 'signs', 'or', 'symptoms', 'of', '[MASK]']
1996 the


Fine Tuned
['[CLS]', 'blood', 'in', 'the', 'urine', ',', 'pain', 'while', 'passing', 'urine', ',', 'pain', 'in', 'the', 'lower', 'back', ',', 'nausea', 'and', 'vomiting', 'are', 'signs', 'or', 'symptoms', 'of', '[MASK]', '[SEP]']
1012 .

Bert


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['blood', 'in', 'urine', ',', 'pain', 'in', 'the', 'left', 'side', 'lower', 'back', ',', 'nausea', 'and', 'vomiting', 'are', 'symptoms', 'of', '[MASK]', 'disease']
1996 the


Fine Tuned
['[CLS]', 'blood', 'in', 'urine', ',', 'pain', 'in', 'the', 'left', 'side', 'lower', 'back', ',', 'nausea', 'and', 'vomiting', 'are', 'symptoms', 'of', '[MASK]', 'disease', '[SEP]']
1996 the

Bert


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['[MASK]', 'pain', ',', 'swelling', 'of', 'the', '[MASK]', ',', 'vomiting', 'are', 'symptoms', 'of', 'app', '##end', '##ici', '##tes', '.']
1012 .
2677 mouth


Fine Tuned
['[CLS]', '[MASK]', 'pain', ',', 'swelling', 'of', 'the', '[MASK]', ',', 'vomiting', 'are', 'symptoms', 'of', 'app', '##end', '##ici', '##tes', '.', '[SEP]']
21419 abdominal
4308 stomach

Bert


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['[MASK]', 'disease', 'will', 'have', 'following', 'symptoms', ',', 'blood', 'in', 'urine', ',', 'pain', 'in', 'the', 'lower', 'back', ',', '[MASK]', 'and', 'vomiting', '.']
1012 .
3255 pain


Fine Tuned
['[CLS]', '[MASK]', 'disease', 'will', 'have', 'following', 'symptoms', ',', 'blood', 'in', 'urine', ',', 'pain', 'in', 'the', 'lower', 'back', ',', '[MASK]', 'and', 'vomiting', '.', '[SEP]']
1996 the
19029 nausea



# Summary :
Fine tuned the BERTforMaskedmodel on the data extracted, trained it for nearly 50 epochs, 
The model failed to guess the masked word in a sentence, but when given with the second name of the disease (removed the first name), it would predict that name, eg. if given 1. [MASK] attack has irregular heatbeat, pain in the arm and chest. The finetuned bertformask would give the word [heart] for masked token, but it fails to predict the whole disease name (heart attack)

# END(work in progress)

In [91]:
data[1]

'what are the symptoms of jaundice? s that later proved to be a manifestation of hemolytic anemia due to g6pd deficiency following vicia faba|fava bean consumption. the main sign of jaundice is a yellowish discoloration of the white area of the eye (sclera) and the skin. choluria|urine is dark in colour. slight increases in serum bilirubin are best detected by examining the sclerae, which have a particular affinity for bilirubin due to their high elastin content. the presence of scleral icterus indicates a serum bilirubin of at least 3 mg/dl. the conjunctiva of the eye are one of the first tissue (biology)|tissues to change color as bilirubin levels rise in jaundice. this is sometimes referred to as \'\'scleral icterus\'\'. the sclera themselves are not "icteric" (stained with bilin (biochemistry)|bile pigment), however, but rather the conjunctival membranes that overlie them. the yellowing of the "white of the eye" is thus more properly termed \'\'conjunctival icterus\'\'. the term "i

In [68]:
import os
# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.txt',
 './model_save/added_tokens.json')

In [ ]:
# Bio BERT

In [30]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
#model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [31]:
#tokenizer.tokenize(text)

In [33]:
tokenizer.max_model_input_sizes

{'TurkuNLP/bert-base-finnish-cased-v1': 512,
 'TurkuNLP/bert-base-finnish-uncased-v1': 512,
 'bert-base-cased': 512,
 'bert-base-cased-finetuned-mrpc': 512,
 'bert-base-chinese': 512,
 'bert-base-german-cased': 512,
 'bert-base-german-dbmdz-cased': 512,
 'bert-base-german-dbmdz-uncased': 512,
 'bert-base-multilingual-cased': 512,
 'bert-base-multilingual-uncased': 512,
 'bert-base-uncased': 512,
 'bert-large-cased': 512,
 'bert-large-cased-whole-word-masking': 512,
 'bert-large-cased-whole-word-masking-finetuned-squad': 512,
 'bert-large-uncased': 512,
 'bert-large-uncased-whole-word-masking': 512,
 'bert-large-uncased-whole-word-masking-finetuned-squad': 512,
 'wietsedv/bert-base-dutch-cased': 512}

In [17]:
model['pooler'].dense()

TypeError: ignored

768

In [18]:
import torch.nn as nn
model.pooler['dense'] = nn.Linear(model.encoder.layer[11].output.dense.out_features)

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)